In [301]:
#이것은 imdb크롤러입니다.

# 패키지 로드
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm #진행상태 확인
import requests
print('패키지 importing...\n' + '-' * 100)

# 크롤링 시점 추출 영역
date = str(datetime.now())
date_1 = date[:date.rfind(':')]
sleep_sec = 0.5

#입력
s_date=input()
e_date=input()

# 셀레니움 제어 시작
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/search/title/?release_date={},{}&sort=release_date,asc&adult=include&country_of_origin=KR".format(s_date, e_date))

time.sleep(2)
driver.maximize_window()

# 스크롤 끝까지 내리기 함수 정의
def scroll_to_bottom():
    # 현재 높이 저장
    prev_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 첫 번째로 스크롤 내리기
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        # 시간 대기
        time.sleep(2)
        # 현재 높이 저장
        current_height = driver.execute_script("return document.body.scrollHeight")
        # 현재 높이와 이전 높이가 같으면 탈출
        if current_height == prev_height:
            break
        # 업데이트해줘서 끝낼 수 있도록
        prev_height = current_height

# 스크롤 끝까지 내리기
scroll_to_bottom()

# "더 보기" 버튼 클릭하고 다시 스크롤 내리기
while True:
    try:
        # "더 보기" 버튼 클릭
        more_button = driver.find_element(By.XPATH, "//span[@class='ipc-see-more__text']")
        more_button.click()
        
        # 시간 대기
        time.sleep(2)
        # 스크롤 끝까지 내리기
        scroll_to_bottom()
    except Exception as e:
        # "더 보기" 버튼이 더 이상 없으면 예외 발생
        break
        
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

driver.quit()

print('검색페이지 호출 완료')

패키지 importing...
----------------------------------------------------------------------------------------------------
2021-07-01
2021-09-31
검색페이지 호출 완료


In [305]:
# 링크 크롤링
idurl_d = soup.find_all('a', {'class': 'ipc-title-link-wrapper'})
idurl = [a.get('href') for a in idurl_d]

# 제목 크롤링
price_d = soup.find_all('h3',{'class': 'ipc-title__text' })
price = [price.text for price in price_d]

# 두 리스트의 길이를 맞추기
min_length = min(len(idurl), len(price))
idurl = idurl[:min_length]
price = price[:min_length]

data = {'title': price,
          'url': idurl}

df = pd.DataFrame(data)

df['title_url'] = 'https://www.imdb.com/' + df['url'].str.split('?').str[0]
df['release_url'] = 'https://www.imdb.com/' + df['url'].str.split('?').str[0] + 'releaseinfo/'

print('주소정보 수집 크롤링 종료')

주소정보 수집 크롤링 종료


In [303]:
df

,title,url,title_url,release_url
0,1. Sweet Curse,/title/tt15151810/?ref_=sr_t_1,https://www.imdb.com//title/tt15151810/,https://www.imdb.com//title/tt15151810/release...
1,2. Ledeu aijeu,/title/tt14979654/?ref_=sr_t_2,https://www.imdb.com//title/tt14979654/,https://www.imdb.com//title/tt14979654/release...
2,3. AleXa: Xtra,/title/tt15011364/?ref_=sr_t_3,https://www.imdb.com//title/tt15011364/,https://www.imdb.com//title/tt15011364/release...
3,4. Gan Tteoleojineun Donggeo,/title/tt13777028/?ref_=sr_t_4,https://www.imdb.com//title/tt13777028/,https://www.imdb.com//title/tt13777028/release...
4,Episode #1.12,/title/tt14728652/?ref_=sr_t_4,https://www.imdb.com//title/tt14728652/,https://www.imdb.com//title/tt14728652/release...
...,...,...,...,...
503,361. Paik's Spirit,/title/tt15381696/?ref_=sr_t_361,https://www.imdb.com//title/tt15381696/,https://www.imdb.com//title/tt15381696/release...
504,362. Twice: The Feels,/title/tt15529432/?ref_=sr_t_362,https://www.imdb.com//title/tt15529432/,https://www.imdb.com//title/tt15529432/release...
505,363. Wasteland,/title/tt15979226/?ref_=sr_t_363,https://www.imdb.com//title/tt15979226/,https://www.imdb.com//title/tt15979226/release...
506,364. Na Honja Sanda,/title/tt7997010/?ref_=sr_t_364,https://www.imdb.com//title/tt7997010/,https://www.imdb.com//title/tt7997010/releasei...


In [304]:
df.to_excel('imdb_20210701_20210931.xlsx', index=False)

In [307]:
t_list = ['https://www.imdb.com//title/tt15151810/']

data2 = []

for r in tqdm(t_list, desc='Processing'):

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)

    url = 'https://www.imdb.com/title/tt15151810/'      
        
    html = response.text
    soup3 = BeautifulSoup(html, 'html.parser')    
    
    #타이틀제목
    name = soup3.find('span', {'data-testid': 'hero__primary-text'})
    names = name.text
    
    #평점
    rating = soup3.find('span', {'class': 'sc-bde20123-1 cMEQkK'})
    ratings = rating.text
    
    #출시연도와 러닝타임       
    t_box = soup3.find('div', {'class': 'sc-69e49b85-0 jqlHBQ'})
    dat = t_box.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'}).text
    
    #장르
    genres = soup3.find('div', {'data-testid': 'genres'}).text
    language = soup3.find('li', {'data-testid': 'title-details-languages'}).text
    
    
    data = {'title_name': names,
            'title_url' : url * len(names),
            'rating': ratings,
            'releaseyear_runningtime': dat,
            'genres': genres,
            'language': language}    
    
    data2.append(data)
    
    
df2 = pd.DataFrame(data2)

Processing: 100%|██| 1/1 [00:01<00:00,  1.92s/it]


In [306]:
names

'Sweet Curse'

In [300]:
df2

,title_name,title_url,rating,releaseyear_runningtime,genres,language
0,Sweet Curse,https://www.imdb.com/title/tt15151810/https://...,5.8,202131m,ShortHorrorRomance,LanguageKorean


In [75]:
r_list = df['release_url'].tolist()
t_list = ['https://www.imdb.com//title/tt15151810/releaseinfo']

In [191]:
data1 = []

for r in tqdm(t_list, desc='Processing'):

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)

    url = 'https://www.imdb.com//title/tt15151810/releaseinfo'      
        
    html = response.text
    soup2 = BeautifulSoup(html, 'html.parser')    
    
    #타이틀 제목
    names = soup2.find_all('h2', {'class': 'sc-a885edd8-9 dcErWY'})
    namess = [name.text for name in names] 
    
    #링크    
    link_tag = soup2.find('link', {'rel': 'canonical'})
    r_url = link_tag.get('href')    
    
    r_div = soup2.find('div', {'data-testid': 'sub-section-releases'})
    
    #출시국가    
    a_tags = r_div.find_all('a', {'class': 'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'})
    r_countries = [a_tag.text for a_tag in a_tags] 
    
    #출시 연월    
    span_tags = r_div.find_all('span', {'class': 'ipc-metadata-list-item__list-content-item'})
    r_date = [span_tag.text for span_tag in span_tags]        
    
    data = {'title_name': namess * len(r_date),
            'release_url': r_url * len(r_date),
            'release_countries': r_countries,
            'release_date': r_date}    
    
    data1.append(data)
    
df = pd.DataFrame(data1)

Processing: 100%|██| 1/1 [00:01<00:00,  1.53s/it]


In [193]:
df

,title_name,release_url,release_countries,release_date
0,[Sweet Curse],https://www.imdb.com/title/tt15151810/releasei...,[South Korea],"[July 1, 2021]"
